In [ ]:
from src.utils import *
import os
import gc
from src.resnet_model import SpectrogramModel
from src.resnet_utils import LoadAttackData_ResNet
from src.resnet_features import compute_spectrum
from sp_utils import recover_mag_spec, retrieve_single_audio
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import librosa
import time
import sys
from attacks.sp_utils import spectrogram_inversion_batch
from attacks_utils import save_perturbed_audio, save_perturbed_spec

In [ ]:
seed_everything(1234)
set_gpu(-1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

config_path = '../config/residualnet_train_config.yaml'
config = read_yaml(config_path)

In [ ]:
df_eval = pd.read_csv(os.path.join('..', config['df_eval_path_3s']))

In [ ]:
model = SpectrogramModel().to(device)
model.load_state_dict(torch.load(os.path.join('..', config['model_path_spec_pow_v0']), map_location=device))

In [ ]:
model.eval()

In [ ]:
epsilon = 3.0
epsilon_str = str(epsilon).replace('.', 'dot')
current_dir = os.path.dirname(os.path.abspath(__file__))

In [ ]:
attack = 'FGSM'   #'FGSM_3s' or 'FGSM'
epsilon = 3.0
dataset = '3s'  # '3s' or 'whole'
model_version = 'v0' # or 'old'
type_of_spec = 'pow'   # 'pow' or 'mag'

In [ ]:
audio_folder = f'FGSM_ResNet_{model_version}_{dataset}_{type_of_spec}_{epsilon_str}'
audio_folder = os.path.join(current_dir, f'FGSM_ResNet_{model_version}_{type_of_spec}', audio_folder)
spec_folder = os.path.join(current_dir, f'FGSM_ResNet_{model_version}_{type_of_spec}', audio_folder, 'spec')

os.makedirs(audio_folder, exist_ok=True)
print(f'Saving the perturbed audio in {audio_folder}...\n')

# data loader
file_eval = list(df_eval['path'])
labels_eval = dict(zip(df_eval['path'], df_eval['label']))

In [ ]:
feat_set = LoadAttackData_ResNet(list_IDs=file_eval,
                                     labels=labels_eval,
                                     win_len=config['win_len'],
                                     config=config,
                                     type_of_spec=type_of_spec)
data_loader = DataLoader(feat_set,
                             batch_size=config['eval_batch_size'],
                             shuffle=False,
                             num_workers=15)

In [ ]:
for batch_x, batch_y, phase, audio_len, index in tqdm(data_loader, total=len(data_loader)):
